## IMPORTS

In [64]:
#imports
from sympy import *
import pandas as pd
import numpy as np
import random
import string
import operator
from sympy.logic.boolalg import is_dnf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import shap
import lime
import lime.lime_tabular
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt

## HELPER FUNCTIONS

In [65]:
# Generate boolean formula using number of variables(string length), number of clauses, length of individual clauses and likelihood of negations
def give_formula(num_string,num_clauses,length_clause,prob_neg):
    complement = ['~','']
    L= list(map(lambda i: ('t' + str(i)), range(num_string)))
    V= list(map(lambda i: Symbol('t' + str(i)), range(num_string)))
    dd = dict(zip(L,V))
    globals().update(dd)
    expression = str() # expression
    condition = True
    while condition:      # Continue to run until generated formula contains all atomic variables we defined in the string
        for i in range(num_clauses):
            variable_list = random.choices(L, k = length_clause)
            local_expression = str()
            for ind,variable in enumerate(variable_list):
                if(ind != length_clause - 1):
                    local_expression += random.choices(complement,cum_weights = [prob_neg,100-prob_neg],k=1)[0] + str(variable) + '&' 
                else:
                    local_expression += random.choices(complement,cum_weights = [prob_neg,100-prob_neg],k=1)[0] + str(variable)
            if(i != num_clauses - 1):
                expression += local_expression + '|'
            else:
                expression += local_expression
        c = all(ele in expression for ele in L) # Check whether formula contain or not all atomic variables
        condition = not c
    dnf_expression = (eval(expression)) 
    return dnf_expression,V

In [66]:
# Create instances based on the size of the dataset and length of the string
def create_instances(size,bits):
    instances = []
    for i in range(size):
        s = np.random.randint(2, size= bits).tolist()     
        instances.append(s)
        instances = [list(y) for y in set([tuple(x) for x in instances])]
    return instances

In [67]:
# Check whether boolean formula is satisfiable or not under the assignment
def satisfiable(instance,tuple_variables,dnf_expression):
    mapping_features = {variable:(True if feature == 1 else False) for feature,variable in zip(instance,tuple_variables)}
    sat = dnf_expression.subs(mapping_features)
    return sat

In [68]:
# Label instances to POS or NEG based on their satisfiability under the assignment
def find_labels(instances,expression,variables):
    labels = []
    for s in instances:
        sat = satisfiable(s,variables,expression)
        labels.append(sat)
    return labels    

In [69]:
# Find each individual clause in the DNF form of the boolean formula
def find_clauses(expr):
    expr = to_dnf(expr,True)
    if not isinstance(expr, Or):
        return expr
    return expr.args

In [70]:
# Prepare the dataframe containing predictor features(binary values of the string) and target feature(satisfiability of the assignment) 
def create_dataframe(size,labels,instances):
    features = [str(feature) for feature in range(1,size+1)]
    df = pd.DataFrame(instances, columns= features)
    df['Target'] = labels
    return df

In [71]:
# Create mapping feeature dictionary to call atomic variables based on their indexes
def map_from_index_to_features(num_string,variables): 
    index_features = [i for i in range(0,num_string)]
    mapping_features = {index:argument for argument,index in zip(variables,index_features)}
    return mapping_features

In [72]:
# Find the maximal explanation accuracy of SHAP
def max_explanation_score_shap(max_explanation_dataset,mapping_features,shap_values,instance_index):
    indexes = []
    for  proposition in max_explanation_dataset[instance_index]: # Find the indexes of the atomic variables in explanation dataset.
        i = list(mapping_features.keys())[list(mapping_features.values()).index(proposition)]
        indexes.append(i)
    len_pro = len(indexes)   # Find the number of atomic variables in the explanation dataset
    shaps = shap_values[1][0].argsort() # Sort the Shapley Values. If you want to use other ensemble tree models like XGBoost or Adaboost, please change 'shap_values[1][0]' part to 'shap_values[0]'
    highest_rankings = shaps[-len_pro:] # Highest k Shap values 
    score = len(list(set(indexes).intersection(highest_rankings)))/len_pro # Find the common atomic variables in explanation dataset and highest k shap values
    return score

In [73]:
# Find the minimal explanation accuracy of SHAP
def min_explanation_score_shap(min_explanation_dataset,mapping_features,shap_values,instance_index):
    indexes = []
    for  proposition in min_explanation_dataset[instance_index]: # Find the indexes of the atomic variables in explanation dataset.
        i = list(mapping_features.keys())[list(mapping_features.values()).index(proposition)]
        indexes.append(i)
    len_pro = len(indexes) # Find the number of atomic variables in the explanation dataset.
    if(len_pro == 0): # If minimal explanation set is empty, we remove it from the general accuracy calculation.
        return 'N'
    else:
        shaps = shap_values[1][0].argsort() # Sort the Shapley Values. If you want to use other ensemble tree models like XGBoost or Adaboost, please change 'shap_values[1][0]' part to 'shap_values[0]'
        highest_rankings = shaps[-len_pro:] # Highest k Shap values 
        score = len(list(set(indexes).intersection(highest_rankings)))/len_pro # Find the common atomic variables in explanation dataset and highest k shap values
        return score

In [74]:
# Find the individual explanation accuracy of SHAP
def indexp_shap(ind_explanation_dataset,mapping_features,shap_values,instance_index):
    scores = []
    for ind_clause in ind_explanation_dataset[instance_index]:
        indexes = []
        for  proposition in ind_clause:
            i = list(mapping_features.keys())[list(mapping_features.values()).index(proposition)]
            indexes.append(i)
            len_pro = len(indexes)   # Find the number of atomic variables in the satisfiable individual clauses
            shaps = shap_values[1][0].argsort() # Sort the Shapley Values. If you want to use other ensemble tree models like XGBoost or Adaboost, please change 'shap_values[1][0]' part to 'shap_values[0]'
            highest_rankings = shaps[-len_pro:] # Highest k Shap values 
            score = len(list(set(indexes).intersection(highest_rankings)))/len_pro
            scores.append(score)
    best = max(scores) # Give the best score
    return best

In [75]:
# Total performance score for the dataset
def total_performance_explanation(truth_table):
    truth_table = [i for i in truth_table if i != 'N'] 
    performance = np.mean(truth_table)
    return performance

In [76]:
# Find the maximal explanation accuracy of LIME
def max_explanation_score_lime(max_explanation_dataset,mapping_features,sorted_list,instance_index):
    indexes = [] 
    for proposition in max_explanation_dataset[instance_index]:  # Find the indexes of the atomic variables in explanation dataset.
        i = list(mapping_features.keys())[list(mapping_features.values()).index(proposition)]
        indexes.append(i)
    len_pro = len(indexes) # Find the number of atomic variables in the explanation dataset.
    sorted_weights = [tuple_index[0] for tuple_index in sorted_list] # Sorted LIME weights
    highest_rankings = sorted_weights[:len_pro]   # Highest k LIME weights  
    score = len(list(set(indexes).intersection(highest_rankings)))/len_pro # Find the common atomic variables in explanation dataset and highest k LIME weights
    return score

In [77]:
# Find the maximal explanation accuracy of LIME
def min_explanation_score_lime(min_explanation_dataset,mapping_features,sorted_list,instance_index):
    indexes = [] 
    for proposition in min_explanation_dataset[instance_index]:  # Find the indexes of the atomic variables in explanation dataset.
        i = list(mapping_features.keys())[list(mapping_features.values()).index(proposition)]
        indexes.append(i)
    len_pro = len(indexes) # Find the number of atomic variables in the explanation dataset.
    if(len_pro == 0):  # If minimal explanation set is empty, we remove it from the general accuracy calculation.
        return 'N'
    else:
        sorted_weights = [tuple_index[0] for tuple_index in sorted_list] # Sorted LIME weights
        highest_rankings = sorted_weights[:len_pro]   # Highest k LIME weights  
        score = len(list(set(indexes).intersection(highest_rankings)))/len_pro # Find the common atomic variables in explanation dataset and highest k LIME weights
        return score

In [78]:
# Find the individual explanation accuracy of SHAP
def indexp_lime(ind_explanation_dataset,mapping_features,sorted_list,instance_index):
    scores = []
    for ind_clause in ind_explanation_dataset[instance_index]:
        indexes = []
        for  proposition in ind_clause:
            i = list(mapping_features.keys())[list(mapping_features.values()).index(proposition)]
            indexes.append(i)
            len_pro = len(indexes)   # Find the number of atomic variables in the satisfiable individual clauses
            sorted_weights = [tuple_index[0] for tuple_index in sorted_list] # Sorted LIME weights
            highest_rankings = sorted_weights[:len_pro]   # Highest k LIME weights
            score = len(list(set(indexes).intersection(highest_rankings)))/len_pro # Find the common atomic variables in explanation dataset and highest k LIME weights
            scores.append(score)
    best = max(scores)
    return best

## BINARY CLASSIFICATION PART

In [79]:
instances = create_instances(5000,25)  # Create instances
expression,variables = expression,variables = give_formula(25,40,6,25) # Generate boolean formula
labels = find_labels(instances,expression,variables) # Label instances POS and NEG based on their satisfiability on boolean formula
clauses = find_clauses(expression)  # Find core asssigments(satisfiable individual clauses in the DNF form of the formula)
mapping_features = map_from_index_to_features(25,variables) # Map atomic variables into the their indexes

In [80]:
max_explanation_dataset = []
min_explanation_dataset = []
labels_dataframe = []
ind_explanation_dataset = []
for i,l in zip(instances,labels):
    if l == False:                             # If the instance is not satisfiable, its explanation is empty. 
        max_explanation_dataset.append(0)
        min_explanation_dataset.append(0)
        ind_explanation_dataset.append(0)
        labels_dataframe.append(0)
    else:                                     # Find the explanation set of the instance which is satisfiable
        labels_dataframe.append(1)
        max_explanation = []
        min_explanation = []
        ind_explanation = []
        t = 0
        for clause in clauses:
            clause_sat = satisfiable(i,variables,clause)       # Check the clause is satisfiable or not
            if clause_sat == True:
                ind_explanation.append(clause.atoms())         # Find individual explanations
                t += 1
                max_explanation.extend(list(clause.atoms()))   # Find maximal explanations
                if(t == 1):
                    min_explanation.extend(list(clause.atoms())) # Find minimal explanations
                else:
                    min_explanation = list(set(min_explanation).intersection(list(clause.atoms())))
            else:
                continue
        max_explanation_dataset.append(list(set(max_explanation)))
        min_explanation_dataset.append(list(set(min_explanation)))
        ind_explanation_dataset.append(ind_explanation)

In [81]:
df = create_dataframe(25,labels_dataframe,instances) # Create dataframe

In [82]:
# Splitting the dataframe into the training and testing sets
y = df['Target']
X = df.drop(['Target'],1)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2 ,random_state=0)

In [ ]:
# Classifier
clf_rf = RandomForestClassifier(n_estimators=150)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)

## EXPLANATION PART - SHAP/LIME

In [84]:
# Treeexplainer for the ensemble model
explainer_rf = shap.TreeExplainer(clf_rf,feature_perturbation = "tree_path_dependent")

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


In [85]:
# Main block for SHAP explanation
y_tst = y_test.tolist()
index_test = []
for items in y_test.iteritems():
    index_test.append(items)
truth_table_shap_max = []
truth_table_shap_min = []
truth_table_shap_ind = []
for i in range(len(y_test)):
    if(y_pred_rf[i] == 1 and y_tst[i] == 1):     #If positive instance is labeled positively by classifier
        instance_index = index_test[i][0]        # Take the index value for the instance
        choosen_instance = X_test.loc[[instance_index]] 
        shap_values = explainer_rf.shap_values(choosen_instance) # Shap value of the instance
        score_shap_max = max_explanation_score_shap(max_explanation_dataset,mapping_features,shap_values,instance_index) # Find the maximal explanation score of SHAP
        score_shap_min = min_explanation_score_shap(min_explanation_dataset,mapping_features,shap_values,instance_index) # Find the minimal explanation score of SHAP
        score_shap_ind = indexp_shap(ind_explanation_dataset,mapping_features,shap_values,instance_index) # Find the individual explanation score of SHAP
        truth_table_shap_max.append(score_shap_max)
        truth_table_shap_min.append(score_shap_min)
        truth_table_shap_ind.append(score_shap_ind)
        performance_shap_max = total_performance_explanation(truth_table_shap_max) # Average maximal explanation score of SHAP
        performance_shap_min = total_performance_explanation(truth_table_shap_min) # Average minimal explanation score of SHAP
        performance_shap_ind = total_performance_explanation(truth_table_shap_ind) # Average individual explanation score of SHAP

In [48]:
# Prepare feature names and target names for LIME explainer
feature_names = X.columns.tolist()
target_names = [0,1]

In [49]:
# LIME explainer for all ML models
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=feature_names, class_names=target_names, discretize_continuous=True)

In [53]:
# Main block for LIME explanation
def take_second(elem):  # Helper function to give the second element of the tuple
    return elem[1]
y_tst = y_test.tolist()
index_test = []
for items in y_test.iteritems():
    index_test.append(items)
truth_table_lime_max = []
truth_table_lime_min = []
truth_table_lime_ind = []
for i in range(len(y_test)):
    if(y_pred_rf[i] == 1 and y_tst[i] == 1):    # If positive instance is labeled positively by classifier
        instance_index = index_test[i][0]       # Take the index value for the instance
        exp = explainer.explain_instance(X_test.loc[instance_index], clf_rf.predict_proba, num_features= X_test.columns.shape[0], top_labels=1)
        weights = exp.as_map()[1]        # Gives feature weights
        sorted_list = sorted(weights, key=take_second,reverse = True)
        score_lime_max = max_explanation_score_lime(max_explanation_dataset,mapping_features,sorted_list,instance_index) # Find the maximal explanation score of SHAP
        score_lime_min = min_explanation_score_lime(min_explanation_dataset,mapping_features,sorted_list,instance_index) # Find the minimal explanation score of SHAP
        score_lime_ind = indexp_lime(ind_explanation_dataset,mapping_features,sorted_list,instance_index) # Find the individual explanation score of SHAP
        truth_table_lime_max.append(score_lime_max)
        truth_table_lime_min.append(score_lime_min)
        truth_table_lime_ind.append(score_lime_ind)
        performance_lime_max = total_performance_explanation(truth_table_lime_max) # Average maximal explanation score of SHAP
        performance_lime_min = total_performance_explanation(truth_table_lime_min) # Average minimal explanation score of SHAP
        performance_lime_ind = total_performance_explanation(truth_table_lime_ind) # Average individual explanation score of SHAP